# NNLS

-- TRAINING --
Height: 3.209 cm
Weight: 1.216 kg

-- TEST --
Height: 3.268 cm
Weight: 1.281 kg

# Taking the mean of symmetric limbs and body parts

- LINEAR REGRESSION

    -LR Error - Height:  2.66

    -LR Error - Weight:  1.07

    -w/o Age
    
    -LR Error - Height:  3.27
    
    -LR Error - Weight:  1.15

- SVR

    -LR Error - Height:  2.89
    
    -LR Error - Weight:  1.05
    

    -w/o Age
    
    -LR Error - Height:  3.41
    
    -LR Error - Weight:  1.11
    

- LASSO

    -LR Error - Height:  2.66
    
    -LR Error - Weight:  1.08

    -w/o Age
    
    -LR Error - Height:  3.26
    
    -LR Error - Weight:  1.15

- Random Forest

    -LR Error - Height:  2.66
    
    -LR Error - Weight:  1.09

    -w/o Age

    -LR Error - Height:  3.24
    
    -LR Error - Weight:  1.14

- XGB

    -LR Error - Height:  2.69
    
    -LR Error - Weight:  1.09

    -w/o Age
    
    -LR Error - Height:  3.18
    
    -LR Error - Weight:  1.16


<img src='importance_height_rf_mean.pdf'>
<img src='importance_weight_rf_mean.pdf'>

<img src='importance_height_lr_mean.pdf'>
<img src='importance_weight_lr_mean.pdf'>

#### LINEAR REGRESSION
---
* LR Error - Height:  2.6679938306918407
* LR Error - Weight:  1.07

##### w/o Age

- LR Error - Height:  3.2793552498050564
- LR Error - Weight:  1.15

#### SVR
---
- LR Error - Height:  2.8965694918927856
- LR Error - Weight:  1.05

##### w/o Age

- LR Error - Height:  3.4110034516469074
- LR Error - Weight:  1.11

#### LASSO
---
- LR Error - Height:  2.6685472242046555
- LR Error - Weight:  1.08

##### w/o Age

- LR Error - Height:  3.2669845918021614
- LR Error - Weight:  1.15

#### Random Forest
---
- LR Error - Height:  2.665557753164557
- LR Error - Weight:  1.09

##### w/o Age

- LR Error - Height:  3.242797789754747 
- LR Error - Weight:  1.14

#### XGB
---
- LR Error - Height:  2.6907522328933595
- LR Error - Weight:  1.09

##### w/o Age

- LR Error - Height:  3.185386208295822
- LR Error - Weight:  1.16

In [1]:
import pandas as pd
from glob import glob
import shutil
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import json
import skimage
import torch

from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn import tree
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from scipy.optimize import nnls

In [2]:
# Read Joint Length and Ages from csv files
length = pd.read_csv('csvs/BODYPIX_FULL_BODY_LENGTH_CM.csv', converters={'name': lambda x: str(x)})
ages = pd.read_csv('csvs/AGES.csv', converters={'ChildID': lambda x: str(x)})

pd.read_csv('csvs/BODYPIX_FULL_BODY_LENGTH_CM.csv').head()

,EYE,HIP,LOWER_ARM_L,LOWER_ARM_R,LOWER_LEG_L,LOWER_LEG_R,SHOULDER,UPPER_ARM_L,UPPER_ARM_R,UPPER_LEG_L,UPPER_LEG_R,name,Age_M
0,1.653573,15.718880,15.152567,17.710094,20.893584,22.762997,15.835947,14.925176,7.700544,23.631376,22.477309,2010305305,43.594471
1,5.384406,10.448997,15.587977,11.264205,16.846044,17.160511,17.405107,13.724348,9.779634,18.526858,18.863811,1010204368,35.276672
2,2.508397,10.665518,12.580524,13.345820,15.100339,17.204327,15.310577,13.014110,12.934076,15.534691,17.361684,1010207351,41.720501
3,2.780574,11.566319,12.085032,14.738835,16.411586,15.219805,16.953852,12.777236,10.222986,17.271757,16.731944,2010305309,39.057491
4,5.509586,9.947918,13.885691,13.917378,14.215253,15.075129,16.114565,15.011198,15.482200,19.133996,20.246705,2010512333,43.298584


In [3]:
# Check the mean joint lengths for each age group

df = pd.merge(left=length, right=ages, left_on='name', right_on='ChildID')
df['Age'] = np.floor(df['Age_M_x'] / 12)
df.groupby(by='Age').mean()

,EYE,HIP,LOWER_ARM_L,LOWER_ARM_R,LOWER_LEG_L,LOWER_LEG_R,SHOULDER,UPPER_ARM_L,UPPER_ARM_R,UPPER_LEG_L,UPPER_LEG_R,Age_M_x,Age_M_y
Age,,,,,,,,,,,,,
2.0,4.141368,9.472442,11.082988,11.618162,13.853100,13.854746,15.663412,12.405842,11.252378,15.775362,16.543152,31.178388,31.178388
3.0,4.314033,10.306391,12.407729,12.992027,15.655966,15.735953,16.496785,13.369308,12.190654,17.566451,18.292219,42.036439,42.036439
4.0,4.481323,10.438786,13.474844,14.105498,17.090326,17.030997,17.361973,14.388533,13.143816,18.547577,19.289815,51.168973,51.168973
5.0,2.883859,12.081401,12.666503,16.209231,20.545386,18.880564,17.501599,14.946632,13.186251,20.331144,21.085336,62.416370,62.416370


In [4]:
# Check the joint length ratios in different age groups

df['UL_ARM_L'] = df['UPPER_ARM_L'] / df['LOWER_ARM_L']
df['UL_ARM_R'] = df['UPPER_ARM_R'] / df['LOWER_ARM_R']

df['UL_LEG_L'] = df['UPPER_LEG_L'] / df['LOWER_LEG_L']
df['UL_LEG_R'] = df['UPPER_LEG_R'] / df['LOWER_LEG_R']

df['ARMLEG_L'] = (df['UPPER_ARM_L'] + df['LOWER_ARM_L']) / (df['UPPER_LEG_L'] + df['LOWER_LEG_L'])
df['ARMLEG_R'] = (df['UPPER_ARM_R'] + df['LOWER_ARM_R']) / (df['UPPER_LEG_R'] + df['LOWER_LEG_R'])

df.groupby(by='Age').mean()[['UL_ARM_L', 'UL_ARM_R', 'UL_LEG_L', 'UL_LEG_R', 'ARMLEG_L', 'ARMLEG_R']]

,UL_ARM_L,UL_ARM_R,UL_LEG_L,UL_LEG_R,ARMLEG_L,ARMLEG_R
Age,,,,,,
2.0,1.175315,1.075528,1.167427,1.223898,0.798101,0.758519
3.0,1.095927,0.967597,1.133456,1.179404,0.779222,0.743659
4.0,1.081810,0.943873,1.098278,1.147538,0.786998,0.756246
5.0,1.190019,0.813201,0.996896,1.120816,0.676295,0.735784


In [5]:
# Standard deviation of joint length ratios in different age groups

df.groupby(by='Age').std()[['UL_ARM_L', 'UL_ARM_R', 'UL_LEG_L', 'UL_LEG_R', 'ARMLEG_L', 'ARMLEG_R']]

,UL_ARM_L,UL_ARM_R,UL_LEG_L,UL_LEG_R,ARMLEG_L,ARMLEG_R
Age,,,,,,
2.0,0.459824,1.473668,0.311409,0.296128,0.112761,0.124606
3.0,0.199235,0.342244,0.153652,0.194792,0.090519,0.095674
4.0,0.171231,0.175755,0.148654,0.170229,0.088457,0.093803
5.0,0.193796,0.007209,0.124375,0.064459,0.027638,0.005676


In [6]:
# Read Limb volumes and lengths

limb_volume = pd.read_csv('csvs/BODYPIX_FULL_BODY_VOLUME_CM2.csv')
limb_length = pd.read_csv('csvs/BODYPIX_FULL_BODY_LENGTH_CM.csv', converters={'name': lambda x: str(x)})
ages = pd.read_csv('csvs/AGES.csv')

In [7]:
# DataFrame Normalizer Function

def normalized_df(df, key, stats):
    
    df = df.reset_index(drop=True).copy()
    
    HEIGHT_MEAN = 91.90
    WEIGHT_MEAN = 12.54
    
    stats[key + '_x_M'] = df[key + '_x'].mean()
    stats[key + '_x_S'] = df[key + '_x'].std()
    
    stats[key + '_y_M'] = df[key + '_y'].mean()
    stats[key + '_y_S'] = df[key + '_y'].std()
    
    df['Height_cm'] = df['Height_cm'] - HEIGHT_MEAN
    df['Weight_kg'] = df['Weight_kg'] - WEIGHT_MEAN
    
    df[key + '_x'] = (df[key + '_x']-df[key + '_x'].mean()) / df[key + '_x'].std()
    df[key + '_y'] = (df[key + '_y']-df[key + '_y'].mean()) / df[key + '_y'].std()
    
    return df


# Test Set Normalizer Function

def normalize_test(df, key, stats):
    
    df = df.reset_index(drop=True).copy()

    df[key + '_x'] = (df[key + '_x']-stats[key + '_x_M']) / stats[key + '_x_S']
    df[key + '_y'] = (df[key + '_y']-stats[key + '_y_M']) / stats[key + '_y_S']
    
    return df


# New normalizer function for separate normalization of body parts
# and limb lengths.

def new_normalizer(X):
    X_new = X.copy()
    
    body_part_std = X.loc[:, body_part_cols].std().mean()
    limb_std = X.loc[:, limb_cols].std().mean()
    
    age_m = X['Age_M'].mean()
    age_std = X['Age_M'].std()
    
    X_new[:][body_part_cols] = X.loc[:, body_part_cols] / body_part_std
    X_new[:][limb_cols] = X.loc[:, limb_cols] / limb_std
    X_new[:]['Age_M'] = (X['Age_M'] - age_m) / age_std
    
    return X_new, body_part_std, limb_std, age_m, age_std

## Training Set Preparation

In [8]:
# Read the training set for regression models.

stats = {}

train_set = pd.concat([pd.read_csv('csvs/REGRESSION_TRAIN.csv', converters={'name': lambda x: str(x)}), 
                       pd.read_csv('csvs/REGRESSION_VAL.csv', converters={'name': lambda x: str(x)})
                      ]).reset_index(drop=True)

# Get the samples where all the features are not zero.

train_set = train_set[(train_set != 0).all(1)]
print('Training set size: %d' % len(train_set))

# Training set features.

print(train_set.columns)

# Get features and labels.

X_df = train_set.loc[:, ~train_set.columns.isin(['name', 'Height_cm', 'Weight_kg'])]
y_df = train_set.loc[:, train_set.columns.isin(['Height_cm', 'Weight_kg'])]

# Set body part and limb length features.

body_part_cols = ['FOOT_L', 'FOOT_R', 'HAND_L', 'HAND_R', 'LOWER_ARM_L_x',
                 'LOWER_ARM_R_x', 'LOWER_LEG_L_x', 'LOWER_LEG_R_x', 'TORSO',
                 'UPPER_ARM_L_x', 'UPPER_ARM_R_x', 'UPPER_LEG_L_x', 'UPPER_LEG_R_x', 'HEAD']

limb_cols = ['EYE', 'HIP', 'LOWER_ARM_L_y', 'LOWER_ARM_R_y', 'LOWER_LEG_L_y', 
             'LOWER_LEG_R_y', 'SHOULDER', 'UPPER_ARM_L_y', 'UPPER_ARM_R_y', 
             'UPPER_LEG_L_y', 'UPPER_LEG_R_y']

# Find standard deviation of body parts and limb lengths.

body_part_std = X_df.loc[:, body_part_cols].std().mean()
limb_std = X_df.loc[:, limb_cols].std().mean()

# Separate normalization.

X_df[:][body_part_cols] = X_df.loc[:, body_part_cols] / body_part_std
X_df[:][limb_cols] = X_df.loc[:, limb_cols] / limb_std
X_df[:]['Age_M'] = (X_df['Age_M'] - X_df['Age_M'].mean()) / X_df['Age_M'].std()

# Take the average of symmetric joints.

X_df['FOOT'] = (X_df['FOOT_L'] + X_df['FOOT_R'])/2
X_df['HAND'] = (X_df['HAND_L'] + X_df['HAND_R'])/2

X_df['LOWER_ARM_x'] = (X_df['LOWER_ARM_L_x'] + X_df['LOWER_ARM_R_x'])/2
X_df['LOWER_ARM_y'] = (X_df['LOWER_ARM_L_y'] + X_df['LOWER_ARM_R_y'])/2

X_df['LOWER_LEG_x'] = (X_df['LOWER_LEG_L_x'] + X_df['LOWER_LEG_R_x'])/2
X_df['LOWER_LEG_y'] = (X_df['LOWER_LEG_L_y'] + X_df['LOWER_LEG_R_y'])/2

X_df['UPPER_ARM_x'] = (X_df['UPPER_ARM_L_x'] + X_df['UPPER_ARM_R_x'])/2
X_df['UPPER_ARM_y'] = (X_df['UPPER_ARM_L_y'] + X_df['UPPER_ARM_R_y'])/2

X_df['UPPER_LEG_x'] = (X_df['UPPER_LEG_L_x'] + X_df['UPPER_LEG_R_x'])/2
X_df['UPPER_LEG_y'] = (X_df['UPPER_LEG_L_y'] + X_df['UPPER_LEG_R_y'])/2

X_df = X_df.drop(columns=['FOOT_L', 'FOOT_R', 'HAND_L', 'HAND_R', 
                   'LOWER_ARM_L_x', 'LOWER_ARM_R_x', 'LOWER_LEG_L_x', 'LOWER_LEG_R_x',
                   'UPPER_ARM_L_x', 'UPPER_ARM_R_x', 'UPPER_LEG_L_x', 'UPPER_LEG_R_x',
                   'LOWER_ARM_L_y', 'LOWER_ARM_R_y', 'LOWER_LEG_L_y', 'LOWER_LEG_R_y', 
                   'UPPER_ARM_L_y', 'UPPER_ARM_R_y', 'UPPER_LEG_L_y', 'UPPER_LEG_R_y'])

X_df.head()

Training set size: 943
Index(['FOOT_L', 'FOOT_R', 'HAND_L', 'HAND_R', 'HEAD', 'LOWER_ARM_L_x',
       'LOWER_ARM_R_x', 'LOWER_LEG_L_x', 'LOWER_LEG_R_x', 'TORSO',
       'UPPER_ARM_L_x', 'UPPER_ARM_R_x', 'UPPER_LEG_L_x', 'UPPER_LEG_R_x',
       'name', 'Height_cm', 'Weight_kg', 'EYE', 'HIP', 'LOWER_ARM_L_y',
       'LOWER_ARM_R_y', 'LOWER_LEG_L_y', 'LOWER_LEG_R_y', 'SHOULDER',
       'UPPER_ARM_L_y', 'UPPER_ARM_R_y', 'UPPER_LEG_L_y', 'UPPER_LEG_R_y',
       'Age_M'],
      dtype='object')


/data/home/can/anaconda3/envs/project/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/data/home/can/anaconda3/envs/project/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,HEAD,TORSO,EYE,HIP,SHOULDER,Age_M,FOOT,HAND,LOWER_ARM_x,LOWER_ARM_y,LOWER_LEG_x,LOWER_LEG_y,UPPER_ARM_x,UPPER_ARM_y,UPPER_LEG_x,UPPER_LEG_y
0,8.162575,15.684949,1.371842,4.124585,7.480960,-1.717071,1.720543,1.040328,1.413779,4.956551,2.734196,6.862450,1.387104,6.356375,3.294373,6.622685
1,8.592276,15.749317,1.042477,3.942539,6.770596,-0.211986,1.435235,1.492645,1.913647,5.828538,2.372923,5.188382,1.530918,4.912141,3.836863,6.510282
2,12.720013,31.205846,0.706490,6.715902,6.765919,0.505314,1.848583,1.650521,2.046646,7.020297,3.741180,9.326151,2.200694,4.833427,5.413708,9.849983
3,9.520041,17.312129,2.353974,4.250254,6.884959,0.467998,1.499732,1.075895,1.597541,5.939440,2.233297,6.257167,1.793158,6.514162,4.059058,8.412715
4,6.183951,12.720013,1.950101,3.969278,6.721432,-1.576099,1.386437,1.111351,1.243392,4.207148,1.804569,5.622683,1.386437,4.774648,2.178687,5.893207


## Test Set Preparation

In [9]:
test_set = pd.read_csv('csvs/REGRESSION_TEST.csv')
test_set = test_set[(test_set != 0).all(1)]

In [10]:
X_df_test = test_set.loc[:, ~test_set.columns.isin(['name', 'Height_cm', 'Weight_kg'])]
X_df_test = (X_df_test - X_df.mean()) / X_df.std()
y_df_test = test_set.loc[:, test_set.columns.isin(['Height_cm', 'Weight_kg'])]

print(len(y_df_test))
y_df_test.head()

316


,Height_cm,Weight_kg
0,91.65,10.4
1,107.05,17.2
2,96.70,14.2
3,83.50,9.8
4,78.50,8.4


In [11]:
HEIGHT_MEAN = 91.90
WEIGHT_MEAN = 12.54

real_h = y_df_test['Height_cm'] - HEIGHT_MEAN
real_w = y_df_test['Weight_kg'] - WEIGHT_MEAN

In [12]:
X_df_test = test_set.loc[:, ~test_set.columns.isin(['name', 'Height_cm', 'Weight_kg'])]

X_df_test[:][body_part_cols] = X_df_test.copy()[body_part_cols] / body_part_std
X_df_test[:][limb_cols] = X_df_test.copy()[limb_cols] / limb_std
X_df_test[:]['Age_M'] = (X_df_test.copy()[['Age_M']] - X_df_test.copy()[['Age_M']].mean()) / X_df_test.copy()[['Age_M']].std()

y_df_test = test_set.loc[:, test_set.columns.isin(['Height_cm', 'Weight_kg'])]

In [13]:
print(len(y_df_test))
print(X_df_test.shape, X_df.shape)
y_df_test.head()

316
(316, 26) (943, 16)


,Height_cm,Weight_kg
0,91.65,10.4
1,107.05,17.2
2,96.70,14.2
3,83.50,9.8
4,78.50,8.4


In [14]:
X_df_test['FOOT'] = (X_df_test['FOOT_L'] + X_df_test['FOOT_R'])/2
X_df_test['HAND'] = (X_df_test['HAND_L'] + X_df_test['HAND_R'])/2

X_df_test['LOWER_ARM_x'] = (X_df_test['LOWER_ARM_L_x'] + X_df_test['LOWER_ARM_R_x'])/2
X_df_test['LOWER_ARM_y'] = (X_df_test['LOWER_ARM_L_y'] + X_df_test['LOWER_ARM_R_y'])/2

X_df_test['LOWER_LEG_x'] = (X_df_test['LOWER_LEG_L_x'] + X_df_test['LOWER_LEG_R_x'])/2
X_df_test['LOWER_LEG_y'] = (X_df_test['LOWER_LEG_L_y'] + X_df_test['LOWER_LEG_R_y'])/2

X_df_test['UPPER_ARM_x'] = (X_df_test['UPPER_ARM_L_x'] + X_df_test['UPPER_ARM_R_x'])/2
X_df_test['UPPER_ARM_y'] = (X_df_test['UPPER_ARM_L_y'] + X_df_test['UPPER_ARM_R_y'])/2

X_df_test['UPPER_LEG_x'] = (X_df_test['UPPER_LEG_L_x'] + X_df_test['UPPER_LEG_R_x'])/2
X_df_test['UPPER_LEG_y'] = (X_df_test['UPPER_LEG_L_y'] + X_df_test['UPPER_LEG_R_y'])/2

X_df_test = X_df_test.drop(columns=['FOOT_L', 'FOOT_R', 'HAND_L', 'HAND_R', 
                   'LOWER_ARM_L_x', 'LOWER_ARM_R_x', 'LOWER_LEG_L_x', 'LOWER_LEG_R_x',
                   'UPPER_ARM_L_x', 'UPPER_ARM_R_x', 'UPPER_LEG_L_x', 'UPPER_LEG_R_x',
                   'LOWER_ARM_L_y', 'LOWER_ARM_R_y', 'LOWER_LEG_L_y', 'LOWER_LEG_R_y', 
                   'UPPER_ARM_L_y', 'UPPER_ARM_R_y', 'UPPER_LEG_L_y', 'UPPER_LEG_R_y'])

/data/home/can/anaconda3/envs/project/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/data/home/can/anaconda3/envs/project/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
print(X_df.shape, X_df_test.shape)

(943, 16) (316, 16)


## Training a Regression Model

In [18]:
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True)

HEIGHT_MEAN = 91.90
WEIGHT_MEAN = 12.54
degree = 2

# Lasso(alpha=0.01)
# SVR(gamma='auto') 
# LinearRegression()
# KNeighborsRegressor(n_neighbors=3) 
# xgb.XGBRegressor(objective ='reg:linear', learning_rate = 0.1, n_estimators=64, reg_alpha=0.1)

model_h = RandomForestRegressor(n_estimators=128)
model_w = RandomForestRegressor(n_estimators=128)

X, y_h, y_w = X_df.to_numpy(), y_df.iloc[:,0].to_numpy()-HEIGHT_MEAN, y_df.iloc[:,1].to_numpy()-WEIGHT_MEAN

cv_mae_h = []
cv_mae_w = []

res = []
np.random.seed(123)

for train_index, val_index in kf.split(X):
        
    model_h.fit(X[train_index], y_h[train_index])
    model_w.fit(X[train_index], y_w[train_index])

    pred_h = model_h.predict(X[val_index])
    pred_w = model_w.predict(X[val_index])

    err_h = mean_absolute_error(y_h[val_index], pred_h)
    err_w = mean_absolute_error(y_w[val_index], pred_w)

    cv_mae_h.append(err_h)
    cv_mae_w.append(err_w)

res.append([np.mean(cv_mae_h), np.mean(cv_mae_w), np.std(cv_mae_h), np.std(cv_mae_w)])

In [19]:
res

[[2.7794653949696757,
  1.098378552255713,
  0.13387150037942638,
  0.06654622892558003]]

In [20]:
res_h = model_h.predict(X_df_test.to_numpy())
res_w = model_w.predict(X_df_test.to_numpy())

print("LR Error - Height: ", mean_absolute_error(real_h, res_h))
print("LR Error - Weight: ", mean_absolute_error(real_w, res_w))

LR Error - Height:  2.660945411392405
LR Error - Weight:  1.095431170886076


## Regression with a Polynomial Model

In [35]:
n_splits = 5
degree = 2

kf = KFold(n_splits=n_splits, shuffle=True)

model_h = make_pipeline(PolynomialFeatures(degree),LinearRegression())
model_w = make_pipeline(PolynomialFeatures(degree),LinearRegression())

X, y_h, y_w = X_df.to_numpy(), y_df.iloc[:,0].to_numpy()-HEIGHT_MEAN, y_df.iloc[:,1].to_numpy()-WEIGHT_MEAN

cv_mae_h = []
cv_mae_w = []

res = []
np.random.seed(123)

for train_index, val_index in kf.split(X):
    
    model_h.fit(X[train_index], y_h[train_index])
    model_w.fit(X[train_index], y_w[train_index])

    pred_h = model_h.predict(X[val_index])
    pred_w = model_w.predict(X[val_index])

    err_h = mean_absolute_error(y_h[val_index], pred_h)
    err_w = mean_absolute_error(y_w[val_index], pred_w)

    cv_mae_h.append(err_h)
    cv_mae_w.append(err_w)

res.append([np.mean(cv_mae_h), np.mean(cv_mae_w), np.std(cv_mae_h), np.std(cv_mae_w)])

In [36]:
res

[[4.766496488433719,
  2.014422130339497,
  0.3913570789759149,
  0.07443967483818043]]

In [37]:
res_h = model_h.predict(X_df_test.to_numpy())
res_w = model_w.predict(X_df_test.to_numpy())

print("LR Error - Height: ", mean_absolute_error(real_h, res_h))
print("LR Error - Weight: ", mean_absolute_error(real_w, res_w))

---

## Multi-Model

### SVR

- Error - Height:  4.05 cm
- Error - Weight:  1.25 kg

Weighted: $exp(- Train Loss)$

- Error - Height:  4.03 cm
- Error - Weight:  1.25 kg

### LR

- Error - Height:  4.28 cm
- Error - Weight:  1.30 kg

Weighted: $exp(- Train Loss)$

- Error - Height:  4.27 cm
- Error - Weight:  1.30 kg

### NN (3)

- Error - Height:  4.08
- Error - Weight:  1.31

Weighted: $exp(- Train Loss)$

- Error - Height:  4.07 cm
- Error - Weight:  1.31 kg


## Single Model

### SVR

- Error - Height:  3.58 cm
- Error - Weight:  1.15 kg

### LR

- Error - Height:  3.54 cm
- Error - Weight:  1.17 kg

### PR - deg=2

- Error - Height:  3.95 kg
- Error - Weight:  1.34 cm

# Age Prediction

### SVM

- Error:  5.53 months

### LR

- Error:  5.63 months

### PR - deg=2

- Error:  6.32 months